In [1]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow.keras as keras
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [2]:
def fine_grained_to_3_class(label):
    if label == '__label__1' or label == '__label__2':
        return 'Negative'
    elif label == '__label__3':
        return 'Neutral'
    return 'Positive'

In [3]:
df_train = pd.read_csv('./sst_train.txt', sep='\t', header=None, names=['truth', 'text'], encoding='ansi')
df_dev = pd.read_csv('./sst_dev.txt', sep='\t', header=None, names=['truth', 'text'], encoding='ansi')
df_test = pd.read_csv('./sst_test.txt', sep='\t', header=None, names=['truth', 'text'], encoding='ansi')

In [4]:
df_train.truth = df_train.truth.apply(fine_grained_to_3_class)
df_test.truth = df_test.truth.apply(fine_grained_to_3_class)
df_dev.truth = df_dev.truth.apply(fine_grained_to_3_class)

In [5]:
embed = hub.Module('D:\\Workspace\\NLP\\tfhub-modules\\google-universal-sentence-encoder')

init = tf.global_variables_initializer()
table_init = tf.tables_initializer()

In [6]:
tf.__version__

'1.15.0-rc2'

In [7]:
x_train = df_train.text.values
y_train = df_train.truth.values

x_dev = df_dev.text.values
y_dev = df_dev.truth.values

x_test = df_test.text.values
y_test = df_test.truth.values

In [8]:
print(" x_train :", x_train.shape)
print(" y _train :", y_train.shape)
print(" x_dev :", x_dev.shape)
print(" y_dev :", y_dev.shape)
print("x_test :", x_test.shape)
print("y_test :", y_test.shape)

 x_train : (8544,)
 y _train : (8544,)
 x_dev : (1101,)
 y_dev : (1101,)
x_test : (2210,)
y_test : (2210,)


In [9]:
le = LabelEncoder()
le.fit(y_train)
y_train = keras.utils.to_categorical(le.transform(y_train))
y_test = keras.utils.to_categorical(le.transform(y_test))
y_dev = keras.utils.to_categorical(le.transform(y_dev))

In [10]:
y_train.shape, y_test.shape, y_dev.shape

((8544, 3), (2210, 3), (1101, 3))

In [11]:
with tf.Session() as sess:
    sess.run([init, table_init])
    x_train = sess.run(embed(x_train))
    x_test = sess.run(embed(x_test))
    x_dev = sess.run(embed(x_dev))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
x_train.shape, x_test.shape, x_dev.shape

((8544, 512), (2210, 512), (1101, 512))

In [13]:
x_train.shape, x_test.shape, x_dev.shape

((8544, 512), (2210, 512), (1101, 512))

In [14]:
_inputs = keras.layers.Input(shape=(512,))

In [15]:
x = keras.layers.Dense(256, activation='relu')(_inputs)
x = keras.layers.Dropout(0.3)(x)
# x = keras.layers.Dense(128, activation='relu')(x)
# x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model = keras.models.Model(_inputs, outputs)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 132,099
Trainable params: 132,099
Non-trainable params: 0
_________________________________________________________________


In [18]:
callbacks = [ keras.callbacks.ReduceLROnPlateau(),
              keras.callbacks.EarlyStopping()
            ]

In [19]:
model.compile('adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

In [20]:
history = model.fit(x_train, y_train, validation_data=(x_dev, y_dev), batch_size=64, epochs=20)

Train on 8544 samples, validate on 1101 samples
Epoch 1/20
8544/8544 [==============================] - 7s 833us/sample - loss: 0.9153 - acc: 0.5961 - val_loss: 0.8545 - val_acc: 0.6322
Epoch 2/20
8544/8544 [==============================] - 1s 91us/sample - loss: 0.8378 - acc: 0.6338 - val_loss: 0.8367 - val_acc: 0.6367
Epoch 3/20
8544/8544 [==============================] - 1s 94us/sample - loss: 0.8238 - acc: 0.6421 - val_loss: 0.8360 - val_acc: 0.6421
Epoch 4/20
8544/8544 [==============================] - 1s 95us/sample - loss: 0.8128 - acc: 0.6430 - val_loss: 0.8431 - val_acc: 0.6412
Epoch 5/20
8544/8544 [==============================] - 1s 96us/sample - loss: 0.8004 - acc: 0.6488 - val_loss: 0.8342 - val_acc: 0.6449
Epoch 6/20
8544/8544 [==============================] - 1s 96us/sample - loss: 0.7904 - acc: 0.6533 - val_loss: 0.8277 - val_acc: 0.6403
Epoch 7/20
8544/8544 [==============================] - 1s 95us/sample - loss: 0.7775 - acc: 0.6594 - val_loss: 0.8257 - val_acc:

In [ ]:
del _inputs
del x
del outputs

In [21]:
model.evaluate(x_test, y_test, verbose=0)

[0.8214253202283005, 0.6466063]

In [22]:
model.save('sst-3-class.hd5')